# Daily workflow - troselab (DRAFT)

`TR TODO: Jupyter ipywidget GUI`

(check Daniel's ipysheet mouse entry GUI for inspiration)

1. Upload to analysis server
    1. automatic (rcopy with file consolidation)?
    2. WinSCP?

    
2. Have this script parse the user folder. 
    1. retrieve login name 
    2. parse user storage folder for SessionIDs, ScanIDs
    3. intersect with 'Session' / 'Scan' tables
    4. FIRST STEP: session ingestion of new data 
        1. have ipywidget with list of data to be ingested with checkbox 
        2. have dropdowns next to SessionsFetch that expose 'Project', 'Anatomical Location', 'Equipment' (NOTE: have me or admin accept new entries? Or user?)
        3. have a checkbox for 'PROCESS ALL SCANS WITH DEFAULT S2P SETTINGS' (needs default s2p per user)
        4. have central 'START' button


3. Scan ingest: 
    1. Start immediately 'PROCESS ALL SCANS' = true (use default s2p settings), or 
    2. have another list with checkboxes - now with all first stage scans. Have dropdown boxes with s2p parameteres. Followed by 'START'
    3. TODO: suppress s2p output, just have progress bars
    

4. Scan Curation
    1. DIFFERENT NOTEBOOK (or different Notebook section)
    2. Make intersection of folder with database
        1. list _all_ scans
        2. have all non-curated red - the others green
        3. bhave non-processed scans grey
        4. have 'CURATE LOCALLY WITH s2P' buttons behind all scans. This button should spawn a local suite2p GUI which directly loads the respective suite2p stats.npy file from the server. Folder settings for server need to be set in datajoint.json file.

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery 
import datajoint as dj
import ipywidgets as widgets

[2023-02-26 16:07:13,975][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-02-26 16:07:14,033][INFO]: Connected tobiasr@172.26.128.53:3306


In [ ]:
db_prefix = dj.config['custom'].get('database.prefix')
print('database Prefix:', db_prefix)

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [2]:
from adamacs.utility import *
from adamacs.nbgui import *
from adamacs.pipeline import subject

Assign easy names for relevant tables

In [3]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

## Entry via GUI
(not implemented currently)

In [ ]:
# """Run this cell to load the subject entry sheet."""
# mouse_gui = MouseEntrySheet()
# mouse_gui.app

In [7]:
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')

In [8]:
import numpy as np
import ipywidgets as widgets

# Read in the numpy arrays
Sessions = np.array(['Session 1', 'Session 2', 'Session 3'])
# Project = np.array(['Project A', 'Project B', 'Project C'])
# Recording_Location = np.array(['Location X', 'Location Y', 'Location Z'])
# Equipment = np.array(['Equipment 1', 'Equipment 2', 'Equipment 3'])

# Define the widgets
session_dropdowns = []
session_checkboxes = []
for session in Sessions:
    # Dropdowns for Project, Recording Location, Equipment
    project_dropdown = widgets.Dropdown(options=Project, value=Project[0])
    location_dropdown = widgets.Dropdown(options=Recording_Location, value=Recording_Location[0])
    equipment_dropdown = widgets.Dropdown(options=Equipment, value=Equipment[0])
    session_dropdowns.append((project_dropdown, location_dropdown, equipment_dropdown))
    # Checkbox for Process?
    session_checkbox = widgets.Checkbox(description='Process?', layout=widgets.Layout(width='auto'))
    session_checkboxes.append(session_checkbox)

# Display the widgets
output = widgets.Output()

with output:
    # Display the Sessions labels and associated dropdowns and checkboxes
    hbox_list = []
    for i, session in enumerate(Sessions):
        # Create an HBox to hold the label and associated dropdowns and checkbox
        hbox = widgets.HBox()
        hbox.children = [
            widgets.Label(value=session + ':', layout=widgets.Layout(width='450px', font_weight='bold')),
            session_dropdowns[i][0],
            session_dropdowns[i][1],
            session_dropdowns[i][2],
            session_checkboxes[i]
        ]
        hbox_list.append(hbox)
    
    vbox = widgets.VBox(hbox_list, layout=widgets.Layout(flex='auto auto auto', overflow_y='scroll'))
    # Display the commit button
    commit_button = widgets.Button(description='Commit', layout=widgets.Layout(width='auto'))
    display(vbox, commit_button)

# Define the callback function for the commit button
def commit_button_clicked(b):
    selected_sessions = [Sessions[i] for i in range(len(Sessions)) if session_checkboxes[i].value]
    selected_projects = [session_dropdowns[i][0].value for i in range(len(Sessions)) if session_checkboxes[i].value]
    selected_locations = [session_dropdowns[i][1].value for i in range(len(Sessions)) if session_checkboxes[i].value]
    selected_equipment = [session_dropdowns[i][2].value for i in range(len(Sessions)) if session_checkboxes[i].value]
    print('Selected Sessions:', selected_sessions)
    print('Selected Projects:', selected_projects)
    print('Selected Recording Locations:', selected_locations)
    print('Selected Equipment:', selected_equipment)

# Attach the callback function to the commit button
commit_button.on_click(commit_button_clicked)

# Display the output
display(output)


Output()

Selected Sessions: []
Selected Projects: []
Selected Recording Locations: []
Selected Equipment: []


In [6]:
# Dropdown example

allequipment = equipment.Equipment().fetch('scanner')

# create a dropdown widget
my_dropdown_equipment = widgets.Dropdown(options=allequipment, value=allequipment[0], layout=widgets.Layout(width='300px'))

# define a function to handle the dropdown selection

print('Select Scanner:')

def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_scanner = change['new']
        print(f"You selected: {selected_scanner}")

# set the dropdown's event handler to our function
test = my_dropdown_equipment.observe(on_dropdown_change)

# display the dropdown widget
display(my_dropdown_equipment)


Select Scanner:


Dropdown(layout=Layout(width='300px'), options=('bench2p', 'Equipment', 'equipment placeholder', 'macroscope',…

## Surgery entry

In [ ]:
data = [{'anatomical_location': 'V1'},
        {'anatomical_location': 'LGNV1'},
        {'anatomical_location': 'RSCa'},
        {'anatomical_location': 'RSCa'},
        {'anatomical_location': 'Ctx'}]
surgery.AnatomicalLocation.insert(data, skip_duplicates=True) 

In [ ]:
allsuregery = surgery.AnatomicalLocation().fetch('anatomical_location')

# create a dropdown widget
my_dropdown_surgery = widgets.Dropdown(options=allsuregery, value=allsuregery[4], layout=widgets.Layout(width='300px'))

# define a function to handle the dropdown selection

print('Select anatomical location:')

def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_anatomical_location = change['new']
        print(f"You selected: {selected_anatomical_location}")

# set the dropdown's event handler to our function
my_dropdown_surgery.observe(on_dropdown_change)

# display the dropdown widget
display(my_dropdown_surgery)


### Project Entry

In [ ]:
# project_key = 'TEC'
# project_description = 'Trace Eyeblink Conditioning'

# project.insert1((project_key, project_description), skip_duplicates=True)

data = [{'project': 'hpc-repstab', 'project_description': 'hpc-representational-stability' },
        {'project': 'vc-lgn-repstab', 'project_description': 'vc-lgn-representational-stability'},
        {'project': 'rsc-functop', 'project_description': 'rsc-functional-topography'},
        {'project': 'rsc-hpc', 'project_description': 'rsc-hippocampal'},
        {'project': 'sc-lgn-actvis', 'project_description': 'sc-lgn-active-vision'},
        {'project': 'rsc-latent', 'project_description': 'rsc-contextual-multimodal-latent-ss'}
        ]
project.insert(data, skip_duplicates=True) 

In [ ]:
project.fetch('project')

In [ ]:
allprojects = project.fetch('project')

# create a dropdown widget
my_dropdown_projects = widgets.Dropdown(options=allprojects, value=allprojects[4], layout=widgets.Layout(width='300px'))

# define a function to handle the dropdown selection

print('Select project:')

def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_project = change['new']
        print(f"You selected: {selected_project}")

# set the dropdown's event handler to our function
my_dropdown_projects.observe(on_dropdown_change)

# display the dropdown widget
display(my_dropdown_projects)


In [ ]:
print(my_dropdown_equipment.value)
print(my_dropdown_surgery.value)
print(my_dropdown_projects.value)

### Line/Mutation

In [ ]:
# POPULATED BY PYRAT INGEST

line_id = 1  # Unique identifier for the line.
line_name = 'Gcamp6-ThyC57BL/6J-Tg(Thy1-GCaMP6s)GP4.12Dkim/J'  # Description of the line.
is_active = 1  # Whether or not the line is actively breeding.

mutation_id = 2  # Unique identifier for the mutation.
mutation_description = 'Tg(Thy1-GCaMP6s)GP4.12Dkim'  # A description of the mutation.

line.insert1((line_id, line_name, is_active), skip_duplicates=True)
mutation.insert1((line_id, mutation_id, mutation_description), skip_duplicates=True)

### Subject

In [ ]:
# POPULATED BY PYRAT INGEST

# subject.Subject.delete()
subject_id = 'Testmouse1'
earmark = ''
sex = 'F'
birth_date = ''
#subject_description = ''
generation = ''
parents = {}
owner_id = 1
responsible_id = 1
line_id = 1
protocol_key = 'LANUF3'

sub.insert1((subject_id, earmark, sex, birth_date,  generation, 
             parents, owner_id, responsible_id, line_id, protocol_key))

In [ ]:
subject = 'WEZ-8701'
earmark = ''
sex = 'M'
birth_date = ''
# subject_description = ''
generation = ''
parents = ''
owner_id = 1
responsible_id = 1
line_id = 100
protocol_key = 'LANUF3'

sub.insert1((subject, earmark, sex, birth_date, generation, 
             parents, owner_id, responsible_id, line_id, protocol_key))

### Subject Genotype

In [ ]:
# POPULATED BY PYRAT INGEST

subject = 'WEZ-8701'
line_id = 1
mutation_id = 2  # Unique identifier for the mutation.
genotype = 'wt/tg'  # The target phenotype of the line.

subject_genotype.insert1((subject, line_id, mutation_id, genotype))

In [ ]:
project.describe()

In [ ]:
project.fetch()

### Subject Death

In [ ]:
subject = 'WEZ-8701'
death_date = '2010-12-01'
case = 'natural'

subject_death.insert1((subject, death_date, case))

## Fetch

### As table

In [ ]:
user * lab

In [ ]:
sub

In [ ]:
line * subject_genotype

In [ ]:
subject_genotype

### As dictionary

One item:

In [ ]:
(sub & 'subject="WEZ-8701"').fetch1()

List of dictionaries:

In [ ]:
(subject_genotype & 'subject="WEZ-8701"').fetch(as_dict=True)

In [ ]:
user